In [1]:
#先引入后面可能用到的包（package）
import pandas as pd  
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
%matplotlib inline   

#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

# 数据准备

In [2]:
import tushare as ts
token='e0eeb08befd1f07516df2cbf9cbd58663f77fd72f92a04f290291c9d'
pro=ts.pro_api(token)

In [3]:
import tushare as ts
def get_data(code,date='20200101'):0.
    data1=ts.pro_bar(ts_code=code, adj='qfq', start_date=date)
    data1=data1[['trade_date','open','high','low','close','vol']]
    data2=pro.daily_basic(ts_code=code,fields='trade_date,turnover_rate,pe,pb')
    data=pd.merge(data1,data2,on='trade_date')
    data.index=pd.to_datetime(data.trade_date)
    data=data.sort_index()
    data['volume']=data.vol
    data['openinterest']=0
    data['datetime']=pd.to_datetime(data.trade_date)
    data=data[['datetime','open','high','low','close',\
               'volume','openinterest','turnover_rate','pe','pb']]
    data=data.fillna(0)
    return data

In [8]:
import tushare as ts
token='315c111b7c5684e65563e52d902c29c130f96437e2726f6336780b71'
pro=ts.pro_api(token)

In [7]:
#数据保存到本地
get_data('300002.SZ').to_csv('test.csv',index=False)
get_data('300002.SZ').head()

请设置tushare pro的token凭证码，如果没有请访问https://tushare.pro注册申请


Exception: api init error.

# 扩展feeds中的数据加载

In [ ]:
#pandas的数据格式
import backtrader as bt
from backtrader.feeds import PandasData
class Addmoredata(PandasData):
    lines = ('turnover_rate','pe','pb',)
    params = (('turnover_rate',7),('pe',8),('pb',9),)

In [ ]:
#直接读取本地csv格式数据
from backtrader.feeds import GenericCSVData
class AddCsvData(GenericCSVData):
    lines = ('turnover_rate','pe','pb',)
    params = (('turnover_rate',7),('pe',8),('pb',9),)

## 添加其他数据——单只股票

In [ ]:
class TestStrategy1(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def next(self):
        self.log(f"换手率:{self.datas[0].turnover_rate[0]},市净率:{self.datas[0].pb[0]},市盈率:{self.datas[0].pe[0]}")

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy1)
feed = Addmoredata(dataname = get_data('300002.SZ','20200420'))
#如果是读取csv数据使用下式
#feed = AddCsvData(dataname = 'test.csv',dtformat=('%Y-%m-%d'))
cerebro.adddata(feed)
cerebro.run()

## 添加其他数据——多只股票

In [ ]:
class TestStrategy2(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def next(self):
        for data in self.datas:     #这里面的datas是所有的data的集合
            print(data._name)
            self.log(f"换手率:{data.turnover_rate[0]},市净率:{data.pb[0]},市盈率:{data.pe[0]}")

In [ ]:
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy2)
codes=['600862.SH','300326.SZ','300394.SZ']
#加载最近两日交易数据
for code in codes:
    feed = Addmoredata(dataname = get_data(code,'20200506'),name=code)
    cerebro.adddata(feed)
cerebro.run()

# 以换手率和市盈率构建交易策略示例

In [ ]:
class MyStrategy(bt.Strategy):

    def next(self):
        if not self.position: # 没有持仓
            if self.datas[0].turnover_rate[0]<3 and 0<self.datas[0].pe[0]<50:
                # 得到当前的账户价值
                total_value = self.broker.getvalue()
                #1手=100股，满仓买入
                ss=int((total_value/100)/self.datas[0].close[0])*100
                self.order=self.buy(size=ss)
        else:
            if self.datas[0].turnover_rate[0]>10 or self.datas[0].pe[0]>80 :
                self.close(self.datas[0])

In [ ]:
cerebro = bt.Cerebro()  
cerebro.addstrategy(MyStrategy)
feed = Addmoredata(dataname = get_data('300002.SZ','20050101'))
cerebro.adddata(feed)
startcash = 100000
cerebro.broker.setcash(startcash) 
cerebro.broker.setcommission(commission=0.001) 
cerebro.run()
portvalue = cerebro.broker.getvalue()
pnl = portvalue - startcash
#打印结果
print(f'期初总资金: {round(startcash,2)}')
print(f'期末总资金: {round(portvalue,2)}')
print(f'净收益: {round(pnl,2)}')

In [ ]:
%matplotlib inline 
cerebro.plot(style='candlestick')

In [ ]:
data = Addmoredata(dataname = get_data('300002.SZ','20050101'))
#注意plot_result是自己写的扩展脚本加入backtrader安装文件里了
#下面直接导入使用
bt.plot_result(MyStrategy,data,startcash = 100000,commission=0.001)

将zjy_plot.py放在backtrader的安装文件夹下（我的路径是：Anaconda3\lib\site-packages\backtrader）,然后使用notepad+（网上下载用于查看py等文件）打开__init__.py文件，在最后加上一句：from .zjy_plot import * 。重新启动即可导入使用。

# 使用pyecharts0.5.11版本对策略指标可视化

In [ ]:
#addmoredata是在PandasData上的扩展
#out_result是自己写的脚本zjy_plot.py里输出策略评价指标的函数
ddf=get_data('300002.SZ','20050101')
data = Addmoredata(dataname = ddf)
df00,df0,df1,df2,df3,df4=bt.out_result(MyStrategy,data,startcash = 100000,commission=0.001)

In [ ]:
def kline_plot(df,name):
    #画K线图数据
    date = df.index.strftime('%Y%m%d').tolist()
    k_value = df[['open','close', 'low','high']].values
    #引入pyecharts画图使用的是0.5.11版本，新版命令需要重写
    kline = Kline(name+'行情走势')
    kline.add('日K线图', date, k_value,
              is_datazoom_show=True,is_splitline_show=False)
    #成交量
    bar = Bar()
    bar.add('成交量', date, df['volume'],tooltip_tragger='axis', 
                is_legend_show=False, is_yaxis_show=False, 
                yaxis_max=5*max(df['volume']))
    overlap = Overlap()
    overlap.add(kline)
    overlap.add(bar,yaxis_index=1, is_add_yaxis=True)
    return overlap    

In [ ]:
kline_plot(ddf,'神州泰岳')

## 评价指标表格

In [ ]:
df00

## 账户价值、持仓市值和收益率

In [ ]:
from pyecharts import*
def plot_result_py(data,v,title,plot_type='line',zoom=False):
    att=data.index
    try:
        attr=att.strftime('%Y%m%d')
    except:
        attr=att
    if plot_type=='line':
        p=Line(title)
        p.add('',attr,list(data[v].round(2)),
         is_symbol_show=False,line_width=2,
        is_datazoom_show=zoom,is_splitline_show=True)
    else:
        p=Bar(title)
        p.add('',attr,[int(i*1000)/10 for i in list(data[v])],
              is_label_show=True,
        is_datazoom_show=zoom,is_splitline_show=True)
    return p

In [ ]:
plot_result_py(df0,'total_value','账户价值')

In [ ]:
plot_result_py(df4,'total_position_value','持仓市值')

In [ ]:
plot_result_py(df3,'year_rate','年化收益%',plot_type='bar')